In [1]:
# MO命令

In [ ]:
mo-->main_onnx-->main-->_convert-->driver-->prepair_ir (生成graph)-->emit_ir
#Graph
    继承了nx.MultiDiGraph，因为要添加的节点有其他的用途，所以要对继承的方法进行重写，并且进行安全检查
    g.graph就是图的本身一些属性
    g.node就是字典了{'节点名字1':{节点属性，节点属性}，'节点名字2':{}}，所以要想找到途中的某个节点，等同于字典查找。所有节点独一无二
    g.edges 是存储节点连接情况的列表字典[(1, 2, {'weight': 10}), ('z', 'h', {})]，字典里保存了边的属性
#Node 
    唯一标识是它的名字id，绑定到图上
    它的属性也是保存在字典中，g['node_id']['attr_1']来进行查找，修改
    {'_in_ports':{0：Port1，1：Port2},'_out_ports':{0：Port1，1：Port2}，'in_ports_count':int,'out_ports_count':int，'kind':'op'}
    ,每个字典中存储的是Node保存的Port,同时，也可以直接用Node来修改Port。
    kind表示类型['in','out']
#Port
    Port(node,idx,'in')
    {'node'= node,'idx' = idx,'type' = type,'data' = self.DataAccessor(),'control_flow' = False,}

In [ ]:
# python 生成MO 的识别Extensions
    OP只是一个属性保存的，使用OP给Node提供属性而已
共三步
1. 将OP加入到Model Optimizer中以便于生成IR文件
2. 在NGraph中执行op，可以云寻
3. 在IE中让不同的设备都可以执行
第一步

    分成三类
1.Model Optimizer operation.基类构建
2.A framework operation extractor.特征提取器
3.front, middle or back 阶段的变形

In [ ]:
命令 mo -->>subprocess_main-->>main_onnx.py-->>main-->>_convert-->>driver----
                    get_onnx_cli_parser解析参数--|


------>>prepare_ir----->>unified_pipeline 
                            建立一个图，对图进行四个操作[load,front,mid,back]
                            get_replacers_order：先对四个操作进行排序
                                _registered_classes_dict全局字典中保存所有类，对他们进行图排序
                    
    
                            apply_replacements_list：遍历图的所有节点，每个类都要
                                    进行对应的find_and_replace_pattern
            
                                    force_clean_up---->>clean_up
                
                                    shape_inference(graph)-->>infer

    
----->>emit_ir

In [ ]:
# mo_extension/ops/cosh.py
from mo.front.common.partial_infer.elemental import copy_shape_infer
from mo.graph.graph import Graph, Node
from mo.ops.op import Op
from mo.ops.pooling import Pooling
import numpy as np
from extensions.ops.argmax import arg_ops_infer


class ArgMinOp(Op):
    version = 'opset1'
    op = 'ArgMin'
    enabled = False                          #表示该OP是否会被MO使用或者包括
    operation = None                         #简单的实现OP方法,写了这个可以不再用C++编写具体实现
    #operation = staticmethod(lambda x: 1 / (1 + np.exp(-x)))   可以直接写出计算OP
    def __init__(self, graph: Graph, attrs: dict):
        mandatory_props = {
            'type':self.op,
            'op': self.op,
            'infer': self.infer,   
            'operation': self.operation,      
            'type_infer': self.type_infer,            #节点的结构属性
            'output_type': np.int64,
            'in_ports_count': 2,
            'out_ports_count': 1,
        }#这些都是默认的,给的是两个attrs字典
        super().__init__(graph, mandatory_props, attrs)
    
    def supported_attrs(self):
        return [
            'top_k',
            'axis',
        ]#这个方法返回列表，其存储的是需要保存到IR中attrs，从上面的初始化中来
    
    @staticmethod
    def infer(node):                      #  计算输出的形状
        """
        静态方法或者类方法，传入的是自身OP构造的Node
        该方法是用来计算输出的形状，类型等张量信息，需要用到node自身的port中所保存data
        node.inport(2).data.get_shape
        或者node.in_node(2).shape
        而data本身包含的六个方法对应处理其属性,
        get_shape()
        set_shape()
        get_value()
        set_value()
        get_attr()
        set_attr()
        value,shape,type
    
        最终的结果就是用这六个方法来计算的得到
        
        通过设置输出port中data的形状完成set_shape，形状是np数组set_value
        
        在这里也可以计算
        shape = x
        value =c
        node.out_port(1).data.set_shape(shape)
        node.out_port(1).data.set_value(value)
        
        node.out_port(0).data.set_shape(node.in_port(0).data.get_shape())
        或者用这种更简单的方法
        node.out_node(0).shape = shape
        node.out_node(0).value = value
        

        """
    @staticmethod
    def type_infer(node):
        """
        类型推断set_data_type
        """
    def backend_attrs(self): #这个是用于保存到IR中的属性，并不是全有
        return [
            ('strides', lambda node: ','.join(map(str, node['stride'][node.spatial_dims]))),
            ('kernel', lambda node: ','.join(map(str, node['window'][node.spatial_dims]))),
            ('pads_begin', lambda node: ','.join(map(str, get_backend_pad(node.pad, node.spatial_dims, 0))))]

In [ ]:
# 2 Operation Extractor
### mo_extension/front/onnx/cosh_ext.py

In [ ]:
from extensions.ops.argmin import ArgMinOp    #要把OP类导入进来，进行属性添加
from mo.front.extractor import FrontExtractorOp
from mo.front.onnx.extractors.utils import onnx_attr

class ArgMinFrontExtractor(FrontExtractorOp):
    op = 'ArgMin'                    #OP名字
    enabled = True                  #MO是否会用到

    @classmethod
    def extract(cls, node):
        #用onnx中的helper函数的onnx_attr来解析onnx模型，读取出OP中的attr，
        #唯一的区别在于不同的框架用不同的解析属性的方法
        keepdims = onnx_attr(node, 'keepdims', 'i', default=1)
        axis = onnx_attr(node, 'axis', 'i', default=0)
        #制作成字典
        attrs = {
            'axis': axis,
            'top_k': 1,
            'keepdims': keepdims,
            'remove_values_output': True
        }
        #将其当作图节点，传入图中对应的节点
        ArgMinOp.update_node_stat(node, attrs)
        return cls.enabled


In [ ]:
!mkdir myop/front/onnx/ -p
%cd myop
!mkdir ops

In [ ]:
%cd /kaggle/working/myop/front/onnx/

file = '''
from openvino.tools.mo.front.extractor import FrontExtractorOp
from myop.ops.zdy import zdyOp

class zdyFrontExtractor(FrontExtractorOp):
    op = 'zdy'
    enabled = True

    @classmethod
    def extract(cls, node):
        
        zdyOp.update_node_stat(node)
        return cls.enabled
'''
with open('zdy_ext.py','w') as f:
    f.write(file)

In [ ]:
%cd /kaggle/working/myop/ops/
file = '''
from openvino.tools.mo.graph.graph import Graph, Node
from openvino.tools.mo.ops.op import Op

class zdyOp(Op):
    op = 'zdy'

    def __init__(self, graph: Graph, attrs: dict):
        super().__init__(graph, {
            'type': __class__.op,
            'op': __class__.op,
            'infer':self.infer,
            'in_ports_count': 1,
            'out_ports_count': 1,
            }, attrs)
    
    @staticmethod
    def infer(node: Node):
        
        input_shape = node.in_port(0).data.get_shape()
        #input_value = node.in_port(0).data.get_value()
        node.out_port(0).data.set_shape(input_shape)
        #node.out_port(0).data.set_value(input_value)
        
'''
with open('zdy.py','w') as f:
    f.write(file)

In [ ]:
%cd /kaggle/working
!mo --input_model test.onnx  --extensions  myop --input_shape [1,2,2,2] 

In [ ]:
会报错的，但是还是生成了.xml文件

# MO的API使用

In [ ]:
from openvino.tools.mo import convert_model

#这个就是MO命令，参数就是MO的接收参数
ov_model = convert_model("resnet.onnx")
